In [31]:
# Only need to run if you want to collect all Data
# There is a file that only get new rows of data 

# FUNCTION:
# WILL COLLECT ALL DATA FROM API ENDPOINT 
# CLEAN, DROP, COLUMNS SO POSTGRESQL WILL BE ABLE TO CAST THE DATA TO ITS REQUIRED DATA TYPE
# EDIT column datatypes in SQL connection 
# DATA LINK: https://data.cityofchicago.org/Community-Economic-Development/Business-Licenses/r5kz-chrr/about_data

In [32]:
import requests
import pandas as pd
from sqlalchemy import create_engine, text
import os

In [33]:
db_pass = os.getenv("db_pass")

dbname="business_owners"
user="postgres"
password=db_pass
host="localhost"
port="5432"

In [34]:
business_licenses_URL = "https://data.cityofchicago.org/resource/r5kz-chrr.json"
LIMIT = 1000

In [35]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [36]:
def fetch_all_data():
    offset = 0
    all_data = []

    while True:
        params = {"$limit": LIMIT, "$offset": offset}
        response = requests.get(business_licenses_URL, params=params)

        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break

        data = response.json()
        if not data:
            break  # Stop when there's no more data

        all_data.extend(data)
        offset += LIMIT
        print(f"Fetched {offset} records...")

    return all_data

In [ ]:
json_data = fetch_all_data()

In [38]:
business_licenses_df = pd.DataFrame(json_data)

In [39]:
# drop misscellaneous columns
business_licenses_df = business_licenses_df.drop(columns=[
    ':@computed_region_vrxf_vc4k',
    ':@computed_region_awaf_s7ux',
    ':@computed_region_6mkv_f3dw',
    ':@computed_region_bdys_3d7i',
    ':@computed_region_43wa_7qmu'
])

In [40]:
# Location from json file doesn't have the right format to cast to POINT data type in POSTGRESQL
# This code modify location column so it has the right format
business_licenses_df['location'] = business_licenses_df.apply(lambda row: f"({row['longitude']},{row['latitude']})", axis=1)

In [ ]:
# Append new rows acquired 
with engine.connect() as conn:
    business_licenses_df.to_sql('business_licenses_uncleaned',conn,if_exists='replace',index=False)
    conn.commit()
    print("Data inserted successfully!")

In [42]:
# Included altering column datatype in here so it only run one file for efficiency
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE business_licenses_uncleaned 
        ALTER COLUMN id TYPE TEXT,
        ALTER COLUMN license_id TYPE NUMERIC(20) USING license_id::NUMERIC(20),
        ALTER COLUMN account_number TYPE NUMERIC(20) USING account_number::NUMERIC(20),
        ALTER COLUMN site_number TYPE NUMERIC(20) USING site_number::NUMERIC(20),
        ALTER COLUMN legal_name TYPE TEXT,
        ALTER COLUMN doing_business_as_name TYPE TEXT,
        ALTER COLUMN address TYPE TEXT,
        ALTER COLUMN city TYPE TEXT,
        ALTER COLUMN state TYPE TEXT,
        ALTER COLUMN zip_code TYPE TEXT,
        ALTER COLUMN ward TYPE NUMERIC(20) USING ward::NUMERIC(20),
        ALTER COLUMN precinct TYPE NUMERIC(20) USING precinct::NUMERIC(20),
        ALTER COLUMN ward_precinct TYPE TEXT,
        ALTER COLUMN police_district TYPE NUMERIC(20) USING police_district::NUMERIC(20),
        ALTER COLUMN community_area TYPE NUMERIC(20) USING community_area::NUMERIC(20),
        ALTER COLUMN community_area_name TYPE TEXT,
        ALTER COLUMN neighborhood TYPE TEXT,
        ALTER COLUMN license_code TYPE NUMERIC(20) USING license_code::NUMERIC(20),
        ALTER COLUMN license_description TYPE TEXT,
        ALTER COLUMN business_activity_id TYPE TEXT,
        ALTER COLUMN business_activity TYPE TEXT,
        ALTER COLUMN license_number TYPE NUMERIC(20) USING license_number::NUMERIC(20),
        ALTER COLUMN application_type TYPE TEXT,
        ALTER COLUMN application_created_date TYPE TIMESTAMP USING application_created_date::TIMESTAMP,
        ALTER COLUMN application_requirements_complete TYPE TIMESTAMP USING application_requirements_complete::TIMESTAMP,
        ALTER COLUMN payment_date TYPE TIMESTAMP USING payment_date::TIMESTAMP,
        ALTER COLUMN conditional_approval TYPE TEXT,
        ALTER COLUMN license_start_date TYPE TIMESTAMP USING license_start_date::TIMESTAMP,
        ALTER COLUMN expiration_date TYPE TIMESTAMP USING expiration_date::TIMESTAMP,
        ALTER COLUMN license_approved_for_issuance TYPE TIMESTAMP USING license_approved_for_issuance::TIMESTAMP,
        ALTER COLUMN date_issued TYPE TIMESTAMP USING date_issued::TIMESTAMP,
        ALTER COLUMN license_status TYPE TEXT,
        ALTER COLUMN license_status_change_date TYPE TIMESTAMP USING license_status_change_date::TIMESTAMP,
        ALTER COLUMN ssa TYPE TEXT,
        ALTER COLUMN latitude TYPE NUMERIC(20,10) USING latitude::NUMERIC(20,10),
        ALTER COLUMN longitude TYPE NUMERIC(20,10) USING longitude::NUMERIC(20,10),
        ALTER COLUMN location TYPE POINT USING location::POINT;
    """))
    conn.commit()